In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


### Webdriver 로드 및 크롬 브라우저 기동

In [2]:
from selenium.webdriver import Chrome
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd

In [3]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized");   # 브라우저 크기 최대화

browser = webdriver.Chrome('chromedriver', options = options)  # 옵션 반영

### 기동된 브라우저를 통한 URL 접속

In [4]:
browser.get('https://www.data.go.kr/')
browser.implicitly_wait(5)  # 묵시적으로 5초 대기(네트워크 속도에 의해 지연될 수 있으므로)

In [5]:
browser.find_element_by_xpath('//*[@id="header"]/div/div/div/div[2]/div/a[1]').click()
browser.implicitly_wait(5)

### ID/Password 입력 및 로그인

In [6]:
browser.find_element_by_xpath('//*[@id="mberId"]').send_keys('wjdtpgml99')

In [7]:
browser.find_element_by_xpath('//*[@id="pswrd"]').send_keys('tpgml586900!')

In [8]:
browser.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
browser.implicitly_wait(5)

### 정보공유 링크 클릭

In [9]:
browser.find_element_by_xpath('//*[@id="M000400_pc"]/a').click()

### 자료실 링크 클릭

In [10]:
browser.find_element_by_xpath('//*[@id="M000402_pc"]/a').click()

### 자료실 데이터 추출 및 저장

In [11]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:   # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists = 'append')
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [12]:
trs = browser.find_elements_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr')
df_list = []
for tr in trs:
    df = pd.DataFrame({
        'NO': [tr.find_element_by_xpath('td[1]').text],
        'TITLE': [tr.find_element_by_xpath('td[2]').text],
        'IQRY': [tr.find_element_by_xpath('td[3]').text],
        'REGDT': [tr.find_element_by_xpath('td[4]').text],
        'CHGDT': [tr.find_element_by_xpath('td[5]').text],
    })
    df_list.append(df)
    
ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)

10 건 저장완료..


### 자료실 글 목록 상세보기 클릭

In [13]:
browser.find_element_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr[1]/td[2]/a').click()
browser.implicitly_wait(3)

### 상세보기 첨부파일 다운로드 및 브라우저 종료

In [14]:
browser.find_element_by_xpath('//*[@id="recsroomDetail"]/div[2]/div[4]/div/a').click()
time.sleep(10)

In [15]:
browser.quit()  # 졸료 안하면 webdriver가 메모리 상에 계속 기동

---

### * 브라우저를 기동하지 않고 background 작업 수행 - headless 기능 이용

In [16]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1280x1024')

browser = webdriver.Chrome('chromedriver', options = options)

In [17]:
browser.get('https://www.data.go.kr/')
browser.implicitly_wait(5)

In [18]:
browser.find_element_by_xpath('//*[@id="header"]/div/div/div/div[2]/div/a[1]').click()
browser.implicitly_wait(5)

In [19]:
browser.find_element_by_xpath('//*[@id="mberId"]').send_keys('wjdtpgml99')

In [20]:
browser.find_element_by_xpath('//*[@id="pswrd"]').send_keys('tpgml586900!')

In [21]:
browser.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()
browser.implicitly_wait(5)

In [23]:
browser.find_element_by_xpath('//*[@id="M000400_pc"]/a').click()

In [24]:
browser.find_element_by_xpath('//*[@id="M000402_pc"]/a').click()

In [25]:
trs = browser.find_elements_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr')
df_list = []
for tr in trs:
    df = pd.DataFrame({
        'NO': [tr.find_element_by_xpath('td[1]').text],
        'TITLE': [tr.find_element_by_xpath('td[2]').text],
        'IQRY': [tr.find_element_by_xpath('td[3]').text],
        'REGDT': [tr.find_element_by_xpath('td[4]').text],
        'CHGDT': [tr.find_element_by_xpath('td[5]').text],
    })
    df_list.append(df)
    
ARTICLE_LIST = pd.concat(df_list)
db_save(ARTICLE_LIST)

10 건 저장완료..


In [26]:
browser.find_element_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr[1]/td[2]/a').click()
browser.implicitly_wait(3)

In [27]:
browser.find_element_by_xpath('//*[@id="recsroomDetail"]/div[2]/div[4]/div/a').click()
time.sleep(10)

In [28]:
browser.quit()